1.0 IMPORTANDO BIBLIOTECAS

In [1]:
import pandas as pd
import requests
from plyer import notification
from datetime import date
import openpyxl
import numpy as np
import json

2.0 CRIANDO ALERTA

In [12]:
def alerta(nivel,base, etapa):
    if nivel == 200:
           nivel = 'API OK'
    else:
           nivel = 'Erro ao acessar a API'

    notification.notify(
        title=nivel,
        message=f'Falha no Carregamento da {base} no endPoint {etapa} no dia {date.today()}',
        app_name='ALERTA',
        timeout=10
        
    )

alerta(2,'API','GET')

3.0 EXTRAÇÃO GERAL

In [13]:
## EXTRAÇÃO DE TODOS OS PAISES 

url = "https://restcountries.com/v3.1/all"
response_01 = requests.get(url)

# testando a conexão e chamando o alerta caso não conecte

if response_01.status_code == 200:
    data_jason_01 = response_01.json()
else:
    alerta(2,'API',url)
    #print("Erroa ao acessar a API")

paises = pd.DataFrame(data_jason_01)
paises.head(5)
#paises.to_excel('paises.xlsx')

,name,tld,cca2,ccn3,cca3,independent,status,unMember,currencies,idd,...,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,postalCode,cioc,borders,gini
0,"{'common': 'Turks and Caicos Islands', 'offici...",[.tc],TC,796,TCA,False,officially-assigned,False,"{'USD': {'name': 'United States dollar', 'symb...","{'root': '+1', 'suffixes': ['649']}",...,[UTC-04:00],[North America],"{'png': 'https://flagcdn.com/w320/tc.png', 'sv...",{},monday,"{'latlng': [21.46, -71.14]}","{'format': 'TKCA 1ZZ', 'regex': '^(TKCA 1ZZ)$'}",NaN,NaN,NaN
1,"{'common': 'Nigeria', 'official': 'Federal Rep...",[.ng],NG,566,NGA,True,officially-assigned,True,"{'NGN': {'name': 'Nigerian naira', 'symbol': '...","{'root': '+2', 'suffixes': ['34']}",...,[UTC+01:00],[Africa],"{'png': 'https://flagcdn.com/w320/ng.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [9.08, 7.53]}","{'format': '######', 'regex': '^(\d{6})$'}",NGR,"[BEN, CMR, TCD, NER]",{'2018': 35.1}
2,"{'common': 'Western Sahara', 'official': 'Sahr...",[.eh],EH,732,ESH,False,officially-assigned,False,"{'DZD': {'name': 'Algerian dinar', 'symbol': '...","{'root': '+2', 'suffixes': ['125288', '125289']}",...,[UTC+00:00],[Africa],"{'png': 'https://flagcdn.com/w320/eh.png', 'sv...",{},monday,"{'latlng': [-13.28, 27.14]}",NaN,NaN,"[DZA, MRT, MAR]",NaN
3,"{'common': 'Taiwan', 'official': 'Republic of ...","[.tw, .台灣, .台湾]",TW,158,TWN,False,officially-assigned,False,"{'TWD': {'name': 'New Taiwan dollar', 'symbol'...","{'root': '+8', 'suffixes': ['86']}",...,[UTC+08:00],[Asia],"{'png': 'https://flagcdn.com/w320/tw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [25.03, 121.52]}","{'format': '#####', 'regex': '^(\d{5})$'}",TPE,NaN,NaN
4,"{'common': 'Costa Rica', 'official': 'Republic...",[.cr],CR,188,CRI,True,officially-assigned,True,"{'CRC': {'name': 'Costa Rican colón', 'symbol'...","{'root': '+5', 'suffixes': ['06']}",...,[UTC-06:00],[North America],"{'png': 'https://flagcdn.com/w320/cr.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [9.93, -84.09]}","{'format': '####', 'regex': '^(\d{4})$'}",CRC,"[NIC, PAN]",{'2019': 48.2}


4.0 TRATANDO A EXTRAÇÃO

In [21]:

# TRAZENDO SOMENTE AS COLUNAS QUE SERÃO USADAS

nome       = [paises['name']['common'] for paises in data_jason_01]
continents = [paises['continents']for paises in data_jason_01]
startOfWeek = [paises['startOfWeek']for paises in data_jason_01]
independent = [paises.get('independent', None) for paises in data_jason_01]
population = paises['population'].astype(int)
region = [paises['region'] for paises in data_jason_01]
subregion = paises['subregion'].astype(str)
area = [paises['area'] for paises in data_jason_01]
currencies = paises['currencies'].astype(str)

paises_tratados = pd.DataFrame({
    "Nome": nome,
    "Continente": continents,
    "Região" : region,
    "subregion" : subregion,
    "area" : area,
    "population" : population,
    "Independente": [str(value).replace('True', 'Verdadeiro').replace('False', 'Falso') for value in independent],
    "Inicio da Semana": [str(value).replace('monday', 'Segunda Feira').replace('tuesday', 'Terça Feira').replace('wednesday', 'Quarta Feira').replace('thursday', 'Quinta Feira').replace('friday', 'Sexta Feira').replace('saturday', 'Sabado').replace('sunday', 'Domingo') for value in startOfWeek],
    "Moedas" : currencies 
    
    
    
    })
paises_tratados.head(2)




,Nome,Continente,Região,subregion,area,population,Independente,Inicio da Semana,Moedas
0,Turks and Caicos Islands,[North America],Americas,Caribbean,948.0,38718,Falso,Segunda Feira,"{'USD': {'name': 'United States dollar', 'symb..."
1,Nigeria,[Africa],Africa,Western Africa,923768.0,206139587,Verdadeiro,Segunda Feira,"{'NGN': {'name': 'Nigerian naira', 'symbol': '..."


In [20]:

#paises.columns
#paises.info()
#paises['subregion']
#paises_tratados.to_excel('paises_tratados.xlsx')


moedas2 = paises['currencies']
#moedas2 = moedas2.to_dict()

moedas2



0      {'USD': {'name': 'United States dollar', 'symb...
1      {'NGN': {'name': 'Nigerian naira', 'symbol': '...
2      {'DZD': {'name': 'Algerian dinar', 'symbol': '...
3      {'TWD': {'name': 'New Taiwan dollar', 'symbol'...
4      {'CRC': {'name': 'Costa Rican colón', 'symbol'...
                             ...                        
245    {'USD': {'name': 'United States dollar', 'symb...
246    {'USD': {'name': 'United States dollar', 'symb...
247    {'LKR': {'name': 'Sri Lankan rupee', 'symbol':...
248    {'ZAR': {'name': 'South African rand', 'symbol...
249    {'NIO': {'name': 'Nicaraguan córdoba', 'symbol...
Name: currencies, Length: 250, dtype: object

In [19]:
for chaves, valores in moedas2.items():
    print(f'Sigla: {chaves}')

    for i, j in valores.items():
        print(f'\t {i} = {j}')

Sigla: 0
	 USD = {'name': 'United States dollar', 'symbol': '$'}
Sigla: 1
	 NGN = {'name': 'Nigerian naira', 'symbol': '₦'}
Sigla: 2
	 DZD = {'name': 'Algerian dinar', 'symbol': 'دج'}
	 MAD = {'name': 'Moroccan dirham', 'symbol': 'DH'}
	 MRU = {'name': 'Mauritanian ouguiya', 'symbol': 'UM'}
Sigla: 3
	 TWD = {'name': 'New Taiwan dollar', 'symbol': '$'}
Sigla: 4
	 CRC = {'name': 'Costa Rican colón', 'symbol': '₡'}
Sigla: 5
	 XOF = {'name': 'West African CFA franc', 'symbol': 'Fr'}
Sigla: 6
	 HNL = {'name': 'Honduran lempira', 'symbol': 'L'}
Sigla: 7
	 USD = {'name': 'United States dollar', 'symbol': '$'}
Sigla: 8
	 AUD = {'name': 'Australian dollar', 'symbol': '$'}
Sigla: 9
	 EUR = {'name': 'Euro', 'symbol': '€'}
Sigla: 10
	 UZS = {'name': 'Uzbekistani soʻm', 'symbol': "so'm"}
Sigla: 11
	 BOB = {'name': 'Bolivian boliviano', 'symbol': 'Bs.'}
Sigla: 12
	 XOF = {'name': 'West African CFA franc', 'symbol': 'Fr'}
Sigla: 13
	 MNT = {'name': 'Mongolian tögrög', 'symbol': '₮'}
Sigla: 14
	 SGD =

AttributeError: 'float' object has no attribute 'items'

In [ ]:
separado = moedas['currencies'].str.split(",")
moedas['currencies'].str.len()


In [ ]:

## TRANSFORMANDO O CONTEUDO DA COLUNA PARA STR
moedas['currencies'] = moedas['currencies'].astype(str)
moedas['currencies'].str.len()



In [61]:

moedas['currencies'].str.len()

separado = moedas['currencies'].str.split(',')


separado.to_excel('SAPARADO.xlsx')

In [73]:
separado2 = np.array(separado)

separado2[2]

#separado2


["{'DZD': {'name': 'Algerian dinar'",
 " 'symbol': 'دج'}",
 " 'MAD': {'name': 'Moroccan dirham'",
 " 'symbol': 'DH'}",
 " 'MRU': {'name': 'Mauritanian ouguiya'",
 " 'symbol': 'UM'}}"]

In [ ]:
'''
moedas['currencies'].str.len()

separado = moedas['currencies'].str.split(",")

separado

#separado.to_excel('separada.xlsx')

#moedas.to_excel('moedas.xlsx')
'''